In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# tensorflow
import tensorflow as tf
from tensorflow import keras

from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Conv1D, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"

In [ ]:
df = pd.read_excel("data/Smart Freeways Data 2018 - Ramps All Days.xlsx",
                   skiprows=3, header=0)
df.head()

In [ ]:
df2 = pd.read_csv('data/clean/LAD.csv')
df2.head()

In [ ]:
df2.DateTime = pd.to_datetime(df2.DateTime)
df2.info()

In [ ]:
df2 = df2[(df2.ID == 1) &
          (df2.DateTime >= '2018-08-27 08:30:00') &
          (df2.DateTime <= '2018-08-27 09:30:00')]

In [ ]:
df2['corridor_id'] = 'KWN'
#df2['demand'] = df2.Volume
df2['volume'] = df2.Volume.astype(int)
df2['speed'] = df2.Speed
df2['time'] = (df2.DateTime - pd.to_datetime('2018-08-27 08:30:00')).astype('timedelta64[m]').astype(int)
df3 = df2[['time', 'corridor_id', 'volume', 'speed']]
df3.reset_index(inplace=True)
df3['time_period'] = tp
df3['date'] = '27/08/2018'
df3 = df3[['corridor_id', 'date', 'time_period', 'volume', 'speed']]
df3

## 27 Aug

In [ ]:
df1 = df[['Row Labels', '27 Aug 18']]
df1['Time'] = df1['Row Labels']
df1 = df1[['Row Labels', 'Time', '27 Aug 18']]
df1.head()

In [ ]:
# For Row Labels, replace time format %H:%M - %H:%M with NaN
# For DateTime, replace everything else as NaN

df1['Row Labels'] = df1['Row Labels'].replace(
    '([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?\s-\s([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?',
    np.nan, regex=True)
df1['Row Labels'] = df1['Row Labels'].ffill(axis=0)
df1 = df1[~df1['Row Labels'].str.contains('KwN ', na=False)]
df1 = df1[~df1['Row Labels'].str.contains('Grand Total', na=False)]
df1['Row Labels'].unique()

In [ ]:
# 27 Aug
OR_df = df1.copy()
OR_df = OR_df[~OR_df['Row Labels'].str.contains('XR', na=False)]
OR_df = OR_df[~OR_df['Row Labels'].str.contains('210', na=False)]
OR_df = OR_df[OR_df['27 Aug 18'].notnull()]
OR_df.Time = OR_df.Time.str[:5]
OR_df = OR_df[OR_df.Time.str.contains(':', na=False)]
OR_df = OR_df[~OR_df['Row Labels'].str.contains('T', na=False)]

In [ ]:
OR_df['Row Labels'].value_counts().sort_index()

In [ ]:
OR_df.Time.value_counts()

In [ ]:
OR_df['Time'] = pd.to_datetime(OR_df['Time'], format='%H:%M')
OR_df = OR_df.melt(id_vars=['Row Labels', 'Time'],
                  var_name='DateTime', value_name='Volume')
OR_df.DateTime = (pd.to_datetime(OR_df['DateTime']) +  
              pd.to_timedelta(OR_df.pop('Time').dt.strftime('%H:%M:%S')))

OR_df = OR_df.sort_values(['Row Labels', 'DateTime'], 
                          ascending=[True, True])
OR_df.reset_index(drop=True, inplace=True)

In [ ]:
# new = OR_df[(OR_df.DateTime >= '2018-08-27 8:30:00') &
#      (OR_df.DateTime <= '2018-08-27 13:30:00')]
new = OR_df[(OR_df.DateTime >= '2018-08-27 8:30:00') &
      (OR_df.DateTime <= '2018-08-27 09:30:00')]
new = pd.pivot_table(new, values='Volume', index='DateTime',
               columns='Row Labels').reset_index()

g = new.set_index('DateTime')
g['0084KWN-ORL'] = g['0084KWN-OR1L'] + g['0084KWN-OR2L']
g['0089KWN-ORL'] = g['0089KWN-OR1L'] + g['0089KWN-OR2L']
g = g[['0081KWN-OR1L', '0084KWN-ORL', '0089KWN-ORL', '0130KWN-OR1',
       '0150KWN-OR1', '0160KWN-OR1L', '0200KWN-OR1L', '0702KWN-OR1L']]
g.columns = ['H500', 'H549', 'H617', 'H547', 'H553', 'H554', 'H622', 'H558']
# g = g*4
g

In [ ]:
dr = pd.date_range('2018-08-27 08:30:00', '2018-08-27 09:30:00', freq='1T')

In [ ]:
newg = g.reindex(dr).interpolate()

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'lines.linewidth': 2})
fig, ax = plt.subplots(figsize=(8,5))
p = sns.lineplot(data=g, ax=ax, linewidth = 2)
ax.legend(loc='upper right', 
               ncol=round(len(g.columns)/4)).set_title('')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
# ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30])) 
ax.set_ylabel('Volume (veh/hr)', fontsize=20)
ax.set_xlabel('Time (HH:MM)', fontsize=18)
ax.grid()
#plt.savefig('fig/27Aug_Ramps_Volume', bbox_inches="tight")
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'lines.linewidth': 2})
fig, ax = plt.subplots(figsize=(8,5))
p = sns.lineplot(data=newg, ax=ax, linewidth = 2)
ax.legend(loc='upper right', 
               ncol=round(len(newg.columns)/4)).set_title('')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
# ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30])) 
ax.set_ylabel('Volume (veh/hr)', fontsize=20)
ax.set_xlabel('Time (HH:MM)', fontsize=18)
plt.savefig('fig/830-930_27Aug_Ramps_Volume', bbox_inches="tight")
plt.show()

In [ ]:
newg = newg.unstack().reset_index(name='demand')
newg.rename(columns={'level_0': 'corridor_id', 'level_1': 'dt'}, inplace=True)

In [ ]:
newg['time'] = (newg.dt - pd.to_datetime('2018-08-27 08:30:00')).astype('timedelta64[m]').astype(int)
newg = newg[['time', 'corridor_id', 'demand']]
newg.demand = np.ceil(newg.demand).astype(int)
newg

In [ ]:
newg = newg.unstack().reset_index(name='volume')
newg.rename(columns={'level_0': 'corridor_id', 'level_1': 'dt'}, inplace=True)
newg.volume = np.ceil(newg.volume).astype(int)

In [ ]:
import datetime as dt
newg['date'] = '27/08/2018'
newg['time_period'] = newg['dt'].dt.strftime('%H%M_') + (newg['dt'] + dt.timedelta(minutes=1)).dt.strftime('%H%M')
newg.time_period = newg.time_period.replace({'0930_0931':'0930_0830'})

In [ ]:
newg.corridor_id.value_counts()

In [ ]:
newg

In [ ]:
#pd.concat([df3, newg]).to_csv('out_data/CaseII_27Aug_demand.csv', index=False)

In [ ]:
#g.to_csv('out_data/27Aug_OR.csv', index=True)

In [ ]:
s = pd.read_csv('data1min/CaseII_supply27Aug1Min.csv')
pd.set_option('display.max_rows', 10)
s.groupby(['link_id_tmc', 'corridor_id', 'corridor_link_order',
           'from_node_id', 'to_node_id', 'geometry', 
           'num_of_lanes', 'reference_speed']).count()

In [ ]:
s.groupby(['link_id_tmc', 'corridor_id', 'corridor_link_order',
           'from_node_id', 'to_node_id', 'geometry', 
           'num_of_lanes', 'reference_speed']).count().iloc[:,0].to_csv('out_data/test.csv')

In [ ]:
s = pd.read_csv('out_data/test.csv')
s.head()

In [ ]:
news = pd.DataFrame(np.repeat(s.values, 61, axis=0),columns=s.columns)
news

In [ ]:
tp = list(newg.time_period.unique())

In [ ]:
from itertools import cycle, islice

it = cycle(tp)
news['time_period'] = list(islice(it, len(news)))
news

In [ ]:
news['date'] = '27/08/2018'
news['travel_time'] = np.nan
news['reference_speed'] = 60
news['queue'] = np.nan
news['notes'] = np.nan

news.info()

In [ ]:
newg = newg[['corridor_id', 'date', 'time_period', 'volume']]
newg

In [ ]:
dat = pd.concat([newg, df3]); dat

In [ ]:
odf = news.merge(dat, on=['corridor_id', 'date', 'time_period'], how='left')

In [ ]:
odf['density'] = np.nan
odf = odf[['link_id_tmc', 'corridor_id', 'corridor_link_order',
           'from_node_id', 'to_node_id', 'time_period', 'date',
           'geometry', 'volume', 'num_of_lanes', 'travel_time',
           'speed', 'reference_speed', 'density', 'queue', 'notes']]
odf.head()

In [ ]:
odf.volume = odf.volume.fillna(87)

In [ ]:
odf.to_csv('out_data/CaseII_27Aug_Supply.csv', index=False)

## 3 Sep

In [ ]:
df1 = df[['Row Labels', '03 Sep 18']]
df1['Time'] = df1['Row Labels']
df1 = df1[['Row Labels', 'Time', '03 Sep 18']]
df1.head()

In [ ]:
# For Row Labels, replace time format %H:%M - %H:%M with NaN
# For DateTime, replace everything else as NaN

df1['Row Labels'] = df1['Row Labels'].replace(
    '([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?\s-\s([01]?[0-9]|2[0-4]):[0-5][0-9](:[0-5][0-9])?',
    np.nan, regex=True)
df1['Row Labels'] = df1['Row Labels'].ffill(axis=0)
df1 = df1[~df1['Row Labels'].str.contains('KwN ', na=False)]
df1 = df1[~df1['Row Labels'].str.contains('Grand Total', na=False)]
df1['Row Labels'].unique()

In [ ]:
# 3 Aug
OR_df = df1.copy()
OR_df = OR_df[~OR_df['Row Labels'].str.contains('XR', na=False)]
OR_df = OR_df[~OR_df['Row Labels'].str.contains('210', na=False)]
OR_df = OR_df[OR_df['03 Sep 18'].notnull()]
OR_df.Time = OR_df.Time.str[:5]
OR_df = OR_df[OR_df.Time.str.contains(':', na=False)]
OR_df = OR_df[~OR_df['Row Labels'].str.contains('T', na=False)]

In [ ]:
OR_df['Row Labels'].value_counts().sort_index()

In [ ]:
OR_df['Time'].value_counts().sort_index()

In [ ]:
OR_df['Time'] = pd.to_datetime(OR_df['Time'], format='%H:%M')
OR_df = OR_df.melt(id_vars=['Row Labels', 'Time'],
                  var_name='DateTime', value_name='Volume')
OR_df.DateTime = (pd.to_datetime(OR_df['DateTime']) +  
              pd.to_timedelta(OR_df.pop('Time').dt.strftime('%H:%M:%S')))

OR_df = OR_df.sort_values(['Row Labels', 'DateTime'], 
                          ascending=[True, True])
OR_df.reset_index(drop=True, inplace=True)

In [ ]:
new = OR_df[(OR_df.DateTime >= '2018-09-03 8:30:00') &
     (OR_df.DateTime <= '2018-09-03 13:30:00')]
new = pd.pivot_table(new, values='Volume', index='DateTime',
               columns='Row Labels').reset_index()

g = new.set_index('DateTime')
g['0084KWN-ORL'] = g['0084KWN-OR1L'] + g['0084KWN-OR2L']
g['0089KWN-ORL'] = g['0089KWN-OR1L'] + g['0089KWN-OR2L']
g = g[['0081KWN-OR1L', '0084KWN-ORL', '0089KWN-ORL', '0130KWN-OR1',
       '0150KWN-OR1', '0160KWN-OR1L', '0200KWN-OR1L', '0702KWN-OR1L']]
g.columns = ['H500', 'H549', 'H617', 'H547', 'H553', 'H554', 'H622', 'H558']
g = g*4
g

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'lines.linewidth': 2})
fig, ax = plt.subplots(figsize=(10,5))
p = sns.lineplot(data=g, ax=ax, linewidth = 2)
ax.legend(loc='upper right', 
               ncol=round(len(g.columns)/2)).set_title('')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30])) 
ax.set_ylabel('Volume (veh/hr)', fontsize=20)
ax.set_xlabel('Time (HH:MM)', fontsize=18)
ax.grid()
#plt.savefig('fig/3Sep/3Sep_Ramps_Volume', bbox_inches="tight")
plt.show()

In [ ]:
g.to_csv('out_data/3Sep_OR.csv', index=True)